# FP-growth算法

**FP-growth的一般流程：**
1. 收集数据：使用任意方法。
2. 准备数据：由于存储的是集合，所以需要离散数据。如果要处理连续数据，需要将它们量化为离散值。
3. 分析数据：使用任意方法。
4. 训练算法：构建一个FP树，并对树进行挖据。
5. 测试算法：没有测试过程。
6. 使用算法：可用于识别经常出现的元素项，从而用于制定决策、推荐元素或进行预测等应用中。

## 创建FP树的数据结构

In [1]:
class treeNode:
    # nameValue: 节点名字，numOccur: 出现次数，parentNode：父节点
    def __init__(self, nameValue, numOccur, parentNode):
        # 节点名字
        self.name = nameValue
        # 计数值
        self.count = numOccur
        # 链接相似的元素项
        self.nodeLink = None
        # 指向当前节点的父节点
        self.parent = parentNode
        # 存放节点的子节点
        self.children = {} 
    
    # 对count增加给定值
    def inc(self, numOccur):
        self.count += numOccur
        
    # 将树以文本形式展示
    def disp(self, ind=1):
        print('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [2]:
# 创建树中的一个单节点
rootNode = treeNode('pyramid', 9, None)

In [3]:
# 为其增加一个子节点
rootNode.children['eye'] = treeNode('eye', 13, None)
rootNode.disp()

   pyramid   9
     eye   13


In [4]:
# 再添加一个节点
rootNode.children['phoenix'] = treeNode('phoenix', 3, None)
rootNode.disp()

   pyramid   9
     eye   13
     phoenix   3


## FP树构建函数

In [5]:
# 构建FP树主函数
# dataSet：字典，其中项集为字典的键，值为频率，minSup：最小支持度
def createTree(dataSet, minSup=1):
    # 头指针表
    headerTable = {}
    # 第一次遍历：统计每个元素项出现的频度
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    headerTableCopy = headerTable.copy()
    # 扫描头指针表，删掉出现次数小于最小支持度的项
    for k in headerTableCopy.keys():
        if headerTable[k] < minSup: 
            del(headerTable[k])
    # 得到频繁项集
    freqItemSet = set(headerTable.keys())
    # 如果所有项都不频繁，就不需要进行下一步处理
    if len(freqItemSet) == 0: return None, None
    # 对头指针表进行扩展，以便可以保存计数值以及指向每种类型第一个元素项的指针
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] 
    # 先创建仅包含空集合的根节点
    retTree = treeNode('Null Set', 1, None)
    # 第二次遍历：构建树
    for tranSet, count in dataSet.items():
        localD = {}
        # 根据全局频率对每个事务中的元素进行排序
        for item in tranSet:
            # 只考虑频繁项集
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            # 使用排序后的频率项集对树进行填充
            updateTree(orderedItems, retTree, headerTable, count)
    return retTree, headerTable

# 填充树函数
# items: 一个项集，inTree: FP树，headerTable: 头指针表，count：频率数
def updateTree(items, inTree, headerTable, count):
    # 事务中的第一个元素项是否作为子节点存在
    if items[0] in inTree.children:
        # 更新该元素项的计数
        inTree.children[items[0]].inc(count)
    else:
        # 如果不存在，创建一个新的treeNode并将其作为一个子节点添加的树中
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        # 更新头指针表
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:
        # 迭代，每次调用都去掉列表中第一个元素
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

# 更新头指针表
def updateHeader(nodeToTest, targetNode):
    # 从头指针表的nodeLink开始，一直沿着nodeLink直到到达链表末尾
    while (nodeToTest.nodeLink != None):
        nodeToTest = nodeToTest.nodeLink
    # 最后再添加最后一个节点
    nodeToTest.nodeLink = targetNode

In [6]:
# 简单数据集及数据包装器
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [7]:
simpDat = loadSimpDat()
simpDat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [8]:
initSet = createInitSet(simpDat)
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [9]:
myFPtree, myHeaderTab = createTree(initSet, 3)

In [10]:
myFPtree.disp()

   Null Set   1
     z   5
       r   1
       x   3
         s   2
           y   2
             t   2
         r   1
           y   1
             t   1
     x   1
       s   1
         r   1


In [11]:
myHeaderTab

{'z': [5, <__main__.treeNode at 0x153f7208>],
 'r': [3, <__main__.treeNode at 0x153f7358>],
 'x': [4, <__main__.treeNode at 0x153f7518>],
 's': [3, <__main__.treeNode at 0x153f7668>],
 'y': [3, <__main__.treeNode at 0x153f76a0>],
 't': [3, <__main__.treeNode at 0x153f70b8>]}

In [12]:
myHeaderTab['x'][1].nodeLink.disp()

   x   1
     s   1
       r   1


## 从一课FP树中挖掘频繁项集

从FP树中抽取频繁项集的三个基本步骤如下：
1. 从FP树中获得条件模式基；
2. 利用条件模式基，构建一个条件FP树；
3. 迭代重复步骤1和步骤2，直到树包含一个元素项为止。

### 抽取条件模式基

In [13]:
# 构建前缀路径，迭代上溯整棵树
def ascendTree(leafNode, prefixPath): 
    if leafNode.parent != None:
        # 加入前缀路径
        prefixPath.append(leafNode.name)
        # 继续上溯迭代
        ascendTree(leafNode.parent, prefixPath)

# 前缀路径
# basePat: 条件模式基，treeNode: 树节点-头指针表
def findPrefixPath(basePat, treeNode): 
    condPats = {}
    while treeNode != None:
        prefixPath = []
        # 得到前缀路径
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            # 去掉所查元素，构建条件模式基
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        # 遍历链表中的下一个节点
        treeNode = treeNode.nodeLink
    return condPats

In [14]:
findPrefixPath('x', myHeaderTab['x'][1])

{frozenset({'z'}): 3}

In [15]:
findPrefixPath('z', myHeaderTab['z'][1])

{}

In [16]:
findPrefixPath('r', myHeaderTab['r'][1])

{frozenset({'z'}): 1, frozenset({'s', 'x'}): 1, frozenset({'x', 'z'}): 1}

### 创建条件FP树

In [17]:
# 递归查找频繁项集的mineTree函数
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    # 对头指针表中的元素项按照出现频率排序
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[0])]
    # 从头指针表的底端开始
    for basePat in bigL:
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        # 创建条件基
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        # 从条件模式基来构建条件FP树
        myCondTree, myHead = createTree(condPattBases, minSup)
        # 如果树中还有元素项，递归调用mineTree()函数
        if myHead != None:
            print('conditional tree for: ',newFreqSet)
            myCondTree.disp(1)            
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

In [18]:
# 建立一个空列表来存储所有的频繁项集
freqItems = []

In [19]:
mineTree(myFPtree, myHeaderTab, 3, set([]), freqItems)

conditional tree for:  {'s'}
   Null Set   1
     x   3
conditional tree for:  {'t'}
   Null Set   1
     z   3
       x   3
         y   3
conditional tree for:  {'x', 't'}
   Null Set   1
     z   3
conditional tree for:  {'t', 'y'}
   Null Set   1
     z   3
       x   3
conditional tree for:  {'x', 't', 'y'}
   Null Set   1
     z   3
conditional tree for:  {'x'}
   Null Set   1
     z   3
conditional tree for:  {'y'}
   Null Set   1
     z   3
       x   3
conditional tree for:  {'x', 'y'}
   Null Set   1
     z   3


In [20]:
freqItems

[{'r'},
 {'s'},
 {'s', 'x'},
 {'t'},
 {'t', 'x'},
 {'t', 'x', 'z'},
 {'t', 'y'},
 {'t', 'x', 'y'},
 {'t', 'x', 'y', 'z'},
 {'t', 'y', 'z'},
 {'t', 'z'},
 {'x'},
 {'x', 'z'},
 {'y'},
 {'x', 'y'},
 {'x', 'y', 'z'},
 {'y', 'z'},
 {'z'}]